Ingeniería de Software Basada en la Nube

#  Unidad 4 - Laboratorio
__________________

## Objetivo

- Construir un primer prototipo de un sistema de software con **arquitectura de microservicios** y orientado a la **nube**.

## Actividades

### Componente #1

* **Tipo de componente:** Base de Datos
* **Nombre:** isbn-users-db
* **Paradigma:** Relacional
* **Sistema de Gestión de Base de Datos:** MySQL

#### Parte 1

Construcción y despliegue local:

**1.** En la consola de GCP acceder a **SQL** > **Create Instance**.

**2.** Seleccionar **MySQL**.

**3.** Crear una instancia con la siguiente información:

* **Instance-ID:** isbn-users-db
* **Password:** 123
* **Database version:** 8.0
* **Cloud SQL edition:** Enterprise
* **Preset:** Sandobox
* **Region:** us-east4
* **Zonal availability:** Single zone

![img1](https://isbn2024i.s3.us-west-2.amazonaws.com/lab3/img/img01.png)

**4.** Acceder a la opción **Databases** y hacer clic en **Create Database**:

* **Database name:** isbn-users-db
* **Character set:** utf8
* **Collation:** Default collation

![img02](https://isbn2024i.s3.us-west-2.amazonaws.com/lab3/img/img02.png)

**5.** Acceder a la opción **Users** y hacer clic en **Add User Account**:

* **User name:** isbn
* **Password:** 123
* **Host name:** Allow any host

![img03](https://isbn2024i.s3.us-west-2.amazonaws.com/lab3/img/img03.png)

**6.** Acceder a la opción **Cloud SQL Studio** y conectarse a la base de datos **isbn-users-db**:

* **Database:** isbn-users-db
* **User:** isbn
* **Password:** 123

![img04](https://isbn2024i.s3.us-west-2.amazonaws.com/lab3/img/img04.png)

**7.**. Crear tabla **User**:

In [ ]:
CREATE TABLE user (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    email VARCHAR(100) NOT NULL
);

![img05](https://isbn2024i.s3.us-west-2.amazonaws.com/lab3/img/img05.png)

**8.** Acceder a la opción **Overview** y hacer clic  en la opción **Edit**.

**9.** Ir a la opción **Connections** > **Authorized networks** > **Add a network**:

* **Name:** Internet
* **Network:**: 0.0.0.0/0

**10.** Hacer clic en **Save**.

### Componente #2

* **Tipo de componente:** Lógico (Microservicio)
* **Nombre:** isbn-users-ms
* **Lenguaje de Programación:** Python
* **Framework:** Flask

**1.** Crear un directorio llamado **isbn-users-ms**.

**2.** Dentro del directorio, crear los siguientes directorios:

    - models/
    - repositories/
    - services/
    - controllers/

**3.** En el directorio **models**, crear archivo **user.py**:

In [ ]:
from flask_sqlalchemy import SQLAlchemy

db = SQLAlchemy()

class User(db.Model):
    
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)
    email = db.Column(db.String(100), nullable=False)

**4.** En el directorio **repositories**, crear archivo **user_repository.py**:

In [ ]:
from models.user import User, db

class UserRepository:

    @staticmethod
    def create_user_repository(name, email):

        user = User(name=name, email=email)
        
        db.session.add(user)
        db.session.commit()

        return user

**5.** En el directorio **services**, crear archivo **user_service.py**:

In [ ]:
from repositories.user_repository import UserRepository

class UserService:
    
    @staticmethod
    def create_user_service(data):

        name = data['name']
        email = data['email']

        return UserRepository.create_user_repository(name, email)

**6.** En el directorio **controllers**, crear archivo **user_controller.py**:

In [ ]:
from flask import Blueprint, request, jsonify
from services.user_service import UserService

user_api = Blueprint('user_api', __name__)

@user_api.route('/api/user', methods=['POST'])
def create_user_controller():

    data = request.get_json()
    UserService.create_user_service(data)

    return jsonify("User has been successfully created."), 201

**7.** En la raíz del proyecto (directorio **isbn-users-ms**), crear archivo **app.py**:

In [ ]:
from flask import Flask
from models.user import db
from controllers.user_controller import *

from config import DB_USER, DB_PASSWORD, DB_HOST, DB_NAME

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'mysql+mysqlconnector://' + DB_USER + ':' + DB_PASSWORD + '@' + DB_HOST + '/' + DB_NAME 
db.init_app(app)

app.register_blueprint(user_api)

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=4000)

**8.** En la raíz del proyecto (directorio **isbn-users-ms**), crear archivo **requirements.txt**:

In [ ]:
Flask==2.3.3
Flask-SQLAlchemy==3.1.1
mysql-connector-python==8.0.27

**9.** En la raíz del proyecto (directorio **isbn-users-ms**), crear archivo **Dockerfile**:

In [ ]:
FROM python

WORKDIR /app

COPY requirements.txt requirements.txt

RUN pip install -r requirements.txt

COPY . .

EXPOSE 4000

CMD ["python", "app.py"]

**10.** En la raíz del proyecto (directorio **isbn-users-ms**), crear archivo **config.py**:

In [ ]:
import os

# Environment Variables (Database)

DB_USER = os.environ.get('DB_USER')
DB_PASSWORD = os.environ.get('DB_PASSWORD')
DB_HOST = os.environ.get('DB_HOST')
DB_NAME = os.environ.get('DB_NAME')

**11.** Crear imagen Docker para el microservicio **isbn-users-ms**:

In [ ]:
docker build -t isbn-users-ms .

**12.** Ejecutar el microservicio **isbn-users-ms** en un contenedor Docker:

*Nota:* **-e** representa la declaración de una variable de entorno en el contexto del contenedor.

*Importante:* reemplazar **IP_ADDRESS** por la IP pública asignada a la instancia de la base de datos creada previamente (**Cloud SQL** > **Overview** > **Connect to this instance** > **Public IP address**).

In [ ]:
docker run -p 4000:4000 -e DB_USER=isbn -e DB_PASSWORD=123 -e DB_HOST=IP_ADDRESS
-e DB_NAME=isbn-users-db --name isbn-users-ms isbn-users-ms

**13.** Consumir la operación expuesta por el microservicio usando un **Cliente HTTP**.

(Herramienta recomendada: [Postman](https://www.postman.com/))

* **Método HTTP:** POST
* **URL:** http://localhost:4000/api/user
* **Cuerpo del mensaje (body):** *(reemplazar los valores con sus datos personales)*.

In [ ]:
{
    "name": "Jeisson",
    "email": "javergarav@unal.edu.co"
}

**14.** Si la respuesta a la petición realizada retorna un código **201** y el mensaje **"User has been successfully created."**, significa que el usuario ha sido creado satisfactoriamente en la base de datos.

**15.** Comprobar el usuario creado en **Cloud SQL** > **Cloud SQL Studio**:

In [ ]:
SELECT * FROM user;

![img06](https://isbn2024i.s3.us-west-2.amazonaws.com/lab3/img/img06.png)

#### Parte 2

Despliegue en Cloud Run:

**1.** En la consola de GCP acceder a **Artifact Registry** > **Create Respository**:

* **Name:** isbn-users-ms
* **Format:** Docker
* **Mode:** Standard
* **Location Type:** Region
* **Region:** us-east4

**2.** En el equipo local, instalar e inicializar [gcloud CLI](https://cloud.google.com/sdk/docs/install). *Nota:* el proceso de inicialización incluye el proceso de autenticación a la cuenta de GCP.

**3.** Subir imagen Docker **isbn-users-ms** a **Artifact Registry**:

In [ ]:
docker tag isbn-users-ms us-east4-docker.pkg.dev/isbn-2024i/isbn-users-ms/isbn-users-ms

In [ ]:
docker push us-east4-docker.pkg.dev/isbn-2024i/isbn-users-ms/isbn-users-ms

**4.** verificar el artefacto subido en **Artifact Registry**:

![img07](https://isbn2024i.s3.us-west-2.amazonaws.com/lab3/img/img07.png)

**5.** En la consola de GCP acceder a **Cloud Run** > **Services** > **Create Service**:

* Deploy one revision from an existing container image.
* **Container image URL:** us-east4-docker.pkg.dev/isbn-2024i/isbn-users-ms > isbn-users-ms > latest
* **Service name**: isbn-users-ms
* **Region:** us-east4
* **Authentication:** Allow unauthenticated invocations
* **CPU allocation and pricing:** CPU is only allocated during request processing
* **Ingress control:** All

Tras la generación del error:

**6** Hacer clic en **Edit & Deploy New Revision**:

* **Container port**: 4000

En **Variables & Secrets**, hacer clic en **Add Variable**, y agregar cada una de las variables definidas en el paso **12** de la **Parte 1**.

**7.** Hacer clic en **Deploy**.

**8.** Repetir los pasos **13**, **14** y **15** de la **Parte 1**, cambiando (en URL) **http://localhost:4000** por la URL asignada para el servicio en Cloud Run.

![img08](https://isbn2024i.s3.us-west-2.amazonaws.com/lab3/img/img08.png)

### Componente #3

* **Tipo de componente:** Base de Datos
* **Nombre:** isbn-tasks-db
* **Paradigma:** NoSQL (Orientado a Documentos)
* **Sistema de Gestión de Base de Datos:** Firestore

**1.** En la consola de GCP acceder a **Firestore** > **Create Database**.

* Native mode
* **Database ID:** isbn-tasks-db
* **Location type:** Region
* **Region:** us-east4
* **Secure rules:** Test rules

![img09](https://isbn2024i.s3.us-west-2.amazonaws.com/lab3/img/img09.png)

**2.** Crear una colección (clic en **Start collection**):

* **Collection ID:** tasks

![img10](https://isbn2024i.s3.us-west-2.amazonaws.com/lab3/img/img10.png)

### Componente #4

* **Tipo de componente:** Lógico (Microservicio)
* **Nombre:** isbn-tasks-ms
* **Lenguaje de Programación:** Python

**1.** En la consola de GCP acceder a **Cloud Functions** > **Create Function**.

* **Environment:** 2nd gen
* **Function name:** isbn-tasks-ms
* **Region:** us-east4
* **Trigger type:** HTTPS
* **Authentication:** Allow unauthenticated invocations
* **Runtime:** Python 3.12
* **Entry point:** tasks_api

Archivo **main.py**:


In [ ]:
import functions_framework

from google.cloud import firestore

db = firestore.Client(project="isbn-2024i", database="isbn-tasks-db")

@functions_framework.http
def tasks_api(request):

    if request.method == 'POST':
        
        data = request.get_json()
    
        doc = db.collection("tasks").document(data['id'])
        doc.set(data['details'])

    return 'Task with id={} created!'.format(data['id']), 201

Archivo **requirements.txt**:

In [ ]:
functions-framework==3.*
google-cloud-firestore==2.16.0

**2.** Clic en **Deploy**.

**3.** Consumir la operación expuesta por el microservicio usando un **Cliente HTTP**.

* **Método HTTP:** POST
* **URL:** URL_ADDRESS
* **Cuerpo del mensaje (body):**

In [ ]:
{
    "id": "abc-123",
    "details": {
        "name": "Tarea 1",
        "description": "Clase de ISBN",
        "user_id": "1"
    }
}

Reemplazar **URL_ADDRESS** (en URL) por la URL asignada para el servicio en Cloud Functions.

![img11](https://isbn2024i.s3.us-west-2.amazonaws.com/lab3/img/img11.png)

**4.** Si la respuesta a la petición realizada retorna un código **201** y el mensaje **"Task with id='' created!"**, significa que la tarea ha sido creada satisfactoriamente en la base de datos.

**5.** Comprobar la tarea creada en **Firestore**:

![img12](https://isbn2024i.s3.us-west-2.amazonaws.com/lab3/img/img12.png)